# Este codigo prepara los datos para usarlos con pytorch (generando un archivo CSV)

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# consts
cats = ["MEL","NV","BCC","AK","BKL","DF","VASC","SCC","UNK"]

train_percent = .7
val_percent = .2
test_percent = .1

## Sacamos los datos del CSV

In [3]:
csv = pd.read_csv("./dataset/ISIC_2019_Training_GroundTruth.csv")

print("> Cantidad de elementos", csv.count(axis=1).size )
print("> Head de los datos del csv")
print(csv.head())

> Cantidad de elementos 25331
> Head de los datos del csv
          image  MEL   NV  BCC   AK  BKL   DF  VASC  SCC  UNK
0  ISIC_0000000  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
1  ISIC_0000001  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
2  ISIC_0000002  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
3  ISIC_0000003  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
4  ISIC_0000004  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0


## Sacamos las categorias (esto no es necesario)

In [4]:
header = list(csv.columns)
header.remove("image")

print("> Categorias")
print(header)

> Categorias
['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']


## Creamos un Dataframe en el formato que requiere pytorch (imagen<string>, categoria<int>)

In [5]:
data = pd.DataFrame({"img":[], "cat": []}, dtype=int)

for entry in csv.values:
    data = data.append({"img":entry[0] , "cat":np.where(entry==1.0)[0][0]-1}, ignore_index=True)


print("> Cantidad de filas", csv.count(axis=1).size)
print("> Head de los datos en el formato que usa pytorch")
print(data.head())

AttributeError: 'DataFrame' object has no attribute 'append'

## Contamos cuantas imagenes hay de cada categoria

In [ ]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

## Averiguamos cual es el tamaño del grupo de imagenes mas pequeño

In [ ]:
# counter = list(filter(lambda elm: elm > 0, counter))
# min_cat_size = min(counter)

min_cat_size = data.cat.value_counts().min()

print("> Cantidad mínima de imagenes de un tipo:")
print(min_cat_size)

## Hacemos que todos los grupos tengan el mismo tamaño, eligiendolos de forma aleatoria

In [ ]:
data = pd.DataFrame(data.groupby("cat").apply(lambda cat: cat.sample(min_cat_size)).reset_index(drop=True))

print("> Cantidad de elemntos:", data.count(axis=1).size )
print("> Datos equilibrados")
print(data.head())

## Barajamos todas las filas del DataFrame

In [ ]:
data = data.sample(frac=1)

print("> cantidad de elementos", data.count(axis=1).size)
print("> datos barajados")
print(data.head())

## Comprobamos cuantas imagenes hay de cada categoria

In [ ]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

## Guardamos el resultado en un archivo CSV
> Esto para mas eficiencia a la hora de usarlo, mantener siempre el mismo dataset y evitar la sobrerrepresentación de las categorias con mas elementos.

In [ ]:
data.to_csv("./dataset/balanced_data.csv", index=False)

## Ahora vamos a separar los datos en 3 grupos (train, val, test)

In [ ]:
train_data, tmp = train_test_split(data, train_size=train_percent, stratify=data['cat'], shuffle=True)
val_data, test_data = train_test_split(tmp, test_size=test_percent/(test_percent+val_percent), stratify=tmp['cat'], shuffle=True)


print("> train", train_data.count(axis=1).size)
print(test_data.head())
print()

print("> val", val_data.count(axis=1).size)
print(val_data.head())
print()

print("> test", test_data.count(axis=1).size)
print(test_data.head())
print()

## Comprobamos cuantas imagenes hay de cada categoria para cada grupo

In [ ]:
counter = np.zeros(len(cats), dtype=int)
for elm in train_data.values:
    counter[int(elm[1])]+=1

print("> train", train_data.count(axis=1).size, train_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/train_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in val_data.values:
    counter[int(elm[1])]+=1

print("> val", val_data.count(axis=1).size, val_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/val_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in test_data.values:
    counter[int(elm[1])]+=1

print("> test", test_data.count(axis=1).size, test_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/test_data.count(axis=1).size})")
print()

## Barajamos los datos

In [ ]:
train_data = train_data.sample(frac=1)
val_data = val_data.sample(frac=1)
test_data = test_data.sample(frac=1)

print("> train", train_data.count(axis=1).size)
print(train_data.head(), end="\n\n")

print("> val", val_data.count(axis=1).size)
print(val_data.head(), end="\n\n")

print("> test", test_data.count(axis=1).size)
print(test_data.head(), end="\n\n")

## Guardamos cada uno de los dataframes en un csv

In [ ]:
train_data.to_csv("dataset/train_data.csv", index=False)
val_data.to_csv("dataset/val_data.csv", index=False)
test_data.to_csv("dataset/test_data.csv", index=False)